In [1]:
# Swin-Unet 训练 & 测试 Notebook（等价原仓库）

# 本 notebook 的目标：

# - 完整复用 https://github.com/HuCaoFighting/Swin-Unet 的代码
# - 在 notebook 中一键完成 **train** 和 **test**
# - 不改动源码逻辑，行为与 `train.py` / `test.py` 完全等价（只是换了运行方式）

# 使用前你需要准备：
# - 有 GPU 的环境（例如：Colab / 自己的服务器）
# - 已下载好作者提供的 **Synapse/BTCV/ACDC** 数据
# - 已下载好 **Swin-T 预训练权重**（放在 `pretrained_ckpt/` 目录下）

# 下面按顺序运行每一个代码单元即可。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
%cd /content/
!pwd
!rm -r /content/Swin-Unet/

/content
/content
rm: cannot remove '/content/Swin-Unet/': No such file or directory


In [23]:
# 如果在 Colab / 服务器上第一次运行，先克隆仓库
!git clone https://github.com/HuCaoFighting/Swin-Unet.git

# 进入仓库目录
%cd Swin-Unet

# 安装依赖（原仓库 requirements）
!pip install -r requirements.txt
!pip install yacs


Cloning into 'Swin-Unet'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 130 (delta 37), reused 22 (delta 22), pack-reused 67 (from 2)
Receiving objects: 100% (130/130), 58.55 KiB | 5.32 MiB/s, done.
Resolving deltas: 100% (53/53), done.
/content/Swin-Unet


In [24]:
import torch, os, sys

print("Python version:", sys.version)
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device count:", torch.cuda.device_count())
    print("Current device:", torch.cuda.current_device())


Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
PyTorch version: 2.8.0+cu126
CUDA available: True
CUDA device count: 1
Current device: 0


In [25]:
# 安装 gdown（Google Drive 下载小工具）
!pip install -q gdown

# 用文件ID下载你的 zip（就是链接中 /d/ 后面的那串）
# 链接: https://drive.google.com/file/d/1BvpY0g9mKkkhdHpAX1HqDw8iTJNbFuwq/view?usp=drive_link
!gdown --id 1BvpY0g9mKkkhdHpAX1HqDw8iTJNbFuwq -O synapse_swinunet.zip

# 看看文件是否下好了
!ls -lh synapse_swinunet.zip

# 解压到当前仓库下的 data/ 目录
!mkdir -p data
!unzip -q synapse_swinunet.zip -d ./data

# 看看解压出来了什么
!ls ./data

# 修改路径
! mv ./lists/lists_Synapse/ ./lists/Synapse/
! cp ./lists/Synapse/test_vol.txt ./lists/Synapse/val.txt

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1BvpY0g9mKkkhdHpAX1HqDw8iTJNbFuwq
From (redirected): https://drive.google.com/uc?id=1BvpY0g9mKkkhdHpAX1HqDw8iTJNbFuwq&confirm=t&uuid=21630988-c26b-41e8-8020-858cc680f20c
To: /content/Swin-Unet/synapse_swinunet.zip
100% 983M/983M [00:04<00:00, 237MB/s]
-rw-r--r-- 1 root root 938M Feb 13  2021 synapse_swinunet.zip
project_TransUNet


In [26]:
import os

# === 数据路径相关 ===
# 1）Synapse 数据：作者的说明里，Synapse 的 npz 训练数据在 root_path/train_npz 下
#    如果你的数据结构和作者保持一致，就把 ROOT_DATA_DIR 设成 train_npz 的上一级目录。
ROOT_DATA_DIR = "/content/Swin-Unet/data/project_TransUNet/data/Synapse"   # 例如："/mnt/data/Synapse"
# 训练脚本里如果 dataset="Synapse"，会自己拼接 "train_npz"，所以这里不用手动加 train_npz。

# 2）测试使用的体数据（.h5），默认在 test.py 里是 ../data/Synapse/test_vol_h5
#    你可以按仓库 README 的结构放数据，也可以自定义路径。
TEST_VOL_DIR = "/content/Swin-Unet/data/project_TransUNet/data/Synapse/test_vol_h5"

# === 输出路径（模型权重、log 等） ===
# OUT_DIR = "/content/Swin-Unet"  # 自己改一个可写的目录
OUT_DIR = "/content/drive/MyDrive/ComputerVisionProject"
os.makedirs(OUT_DIR, exist_ok=True)

# === Swin-Unet 的配置文件 ===
CFG_FILE = "configs/swin_tiny_patch4_window7_224_lite.yaml"

# === 训练超参数（跟 README 示例保持一致） ===
IMG_SIZE   = 224
BATCH_SIZE = 24   # 显存不够就改成 12 或 6
MAX_EPOCHS = 150
BASE_LR    = 0.05
N_CLASS  = 9

print("ROOT_DATA_DIR:", ROOT_DATA_DIR)
print("TEST_VOL_DIR :", TEST_VOL_DIR)
print("OUT_DIR      :", OUT_DIR)
print("CFG_FILE     :", CFG_FILE)


print("IMG_SIZE     :", IMG_SIZE)
print("BATCH_SIZE   :", BATCH_SIZE)
print("MAX_EPOCHS   :", MAX_EPOCHS)
print("N_CLASS      :", N_CLASS)

ROOT_DATA_DIR: /content/Swin-Unet/data/project_TransUNet/data/Synapse
TEST_VOL_DIR : /content/Swin-Unet/data/project_TransUNet/data/Synapse/test_vol_h5
OUT_DIR      : /content/drive/MyDrive/ComputerVisionProject
CFG_FILE     : configs/swin_tiny_patch4_window7_224_lite.yaml
IMG_SIZE     : 224
BATCH_SIZE   : 24
MAX_EPOCHS   : 150
N_CLASS      : 9


In [27]:
PRETRAIN_DIR = "pretrained_ckpt"
os.makedirs(PRETRAIN_DIR, exist_ok=True)

# 下载预训练模型
!gdown --id 1TyMf0_uvaxyacMmVzRfqvLLAWSOE2bJR
! mv swin_tiny_patch4_window7_224.pth ./pretrained_ckpt

print("预训练权重目录:", os.path.abspath(PRETRAIN_DIR))
print("预训练权重文件列表:", os.listdir(PRETRAIN_DIR))


/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1TyMf0_uvaxyacMmVzRfqvLLAWSOE2bJR
From (redirected): https://drive.google.com/uc?id=1TyMf0_uvaxyacMmVzRfqvLLAWSOE2bJR&confirm=t&uuid=abd43aca-9d59-4409-8a47-2640ba315860
To: /content/Swin-Unet/swin_tiny_patch4_window7_224.pth
100% 114M/114M [00:00<00:00, 172MB/s] 
预训练权重目录: /content/Swin-Unet/pretrained_ckpt
预训练权重文件列表: ['swin_tiny_patch4_window7_224.pth']


In [ ]:
# 注意：下面命令与 README 中示例一致，只是把路径用我们上面定义的变量代替
!python train.py \
    --dataset Synapse \
    --cfg {CFG_FILE} \
    --root_path {ROOT_DATA_DIR} \
    --max_epochs {MAX_EPOCHS} \
    --output_dir {OUT_DIR} \
    --img_size {IMG_SIZE} \
    --base_lr {BASE_LR} \
    --batch_size {BATCH_SIZE} \
    --n_class {N_CLASS} \
    --resume True

Streaming output truncated to the last 5000 lines.
Val: 8:  53% 49/93 [00:29<00:37,  1.16it/s]
Val: 8:  54% 50/93 [00:29<00:28,  1.50it/s]
Val: 8:  55% 51/93 [00:29<00:22,  1.89it/s]
Val: 8:  56% 52/93 [00:29<00:17,  2.32it/s]
Val: 8:  57% 53/93 [00:30<00:14,  2.76it/s]
Val: 8:  58% 54/93 [00:30<00:12,  3.21it/s]
Val: 8:  59% 55/93 [00:30<00:10,  3.60it/s]
Val: 8:  60% 56/93 [00:30<00:09,  3.97it/s]
Val: 8:  61% 57/93 [00:32<00:24,  1.49it/s]
Val: 8:  62% 58/93 [00:32<00:18,  1.89it/s]
Val: 8:  63% 59/93 [00:32<00:14,  2.31it/s]
Val: 8:  65% 60/93 [00:33<00:12,  2.74it/s]
Val: 8:  66% 61/93 [00:33<00:10,  3.06it/s]
Val: 8:  67% 62/93 [00:33<00:08,  3.46it/s]
Val: 8:  68% 63/93 [00:33<00:07,  3.77it/s]
Val: 8:  69% 64/93 [00:33<00:07,  4.04it/s]
Val: 8:  70% 65/93 [00:36<00:28,  1.00s/it]
Val: 8:  71% 66/93 [00:36<00:20,  1.29it/s]
Val: 8:  72% 67/93 [00:37<00:16,  1.61it/s]
Val: 8:  73% 68/93 [00:37<00:13,  1.92it/s]
Val: 8:  74% 69/93 [00:37<00:10,  2.27it/s]
Val: 8:  75% 70/93 [00:37